In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import sqlite3

In [2]:
log_file = 'code_log.txt'
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
attribute_list = ['Name', 'MC_USD_Billion']
csv_path = 'Largest_banks.csv'
database_name = 'Banks.db'
table_name = 'Largest_banks'

In [3]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%m:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, 'a') as f:
        f.write(timestamp + ": " + message + "\n")

In [19]:
def extract(url, attributes):
    df = pd.DataFrame(columns=attributes)
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')

    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            data_dict = {
                'Name': col[1].find_all('a')[1].get('title'),
                'MC_USD_Billion': float(col[2].contents[0].split("\n")[0])
            }

            new_df = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, new_df], ignore_index=True)

    return df

In [5]:
def transform(df):
    df_copied = df.copy()
    df_copied['MC_EUR_Billion'] = np.round(df['MC_USD_Billion'] * 0.92, 2)
    df_copied['MC_GBP_Billion'] = np.round(df['MC_USD_Billion'] * 0.77, 2)
    df_copied['MC_INR_Billion'] = np.round(df['MC_USD_Billion'] * 84.08, 2)

    return df_copied

In [6]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

def load_to_db(df, table_name, connect):
    df.to_sql(table_name, connect, if_exists='replace', index=False)

In [7]:
def run_query(query, connect):
    output_query = pd.read_sql(query, connect)
    print(query)
    print(output_query)

In [20]:
# Start Project

log_progress('Preliminaries complete. Initiating extract data.')

df = extract(url, attribute_list)
log_progress('Extraction complete. Initiating transform data.')

transformed_df = transform(df)
log_progress('Transformation complete. Initiating load to csv.')

load_to_csv(transformed_df, csv_path)
log_progress('Load to csv complete. Initiating load to database.')

conn = sqlite3.connect(database_name)
log_progress('SQL Connection initiated')

load_to_db(transformed_df, table_name, conn)
log_progress('Load to database complete. Intiating query.')


query_statements = [
    'SELECT * FROM Largest_banks',
    'SELECT AVG(MC_GBP_Billion) FROM Largest_banks',
    'SELECT Name from Largest_banks LIMIT 5'
]
for sql in query_statements:
  run_query(sql, conn)
log_progress('Process Complete')

conn.close()
log_progress('Server Connection closed')

SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          398.29   
1                          Bank of America          231.52          213.00   
2  Industrial and Commercial Bank of China          194.56          179.00   
3               Agricultural Bank of China          160.68          147.83   
4                                HDFC Bank          157.91          145.28   
5                              Wells Fargo          155.87          143.40   
6                                     HSBC          148.90          136.99   
7                           Morgan Stanley          140.83          129.56   
8                  China Construction Bank          139.82          128.63   
9                            Bank of China          136.81          125.87   

   MC_GBP_Billion  MC_INR_Billion  
0          333.35        36399.91  
1          178.27        19466.20  
2    

C:\Users\oab13\AppData\Local\Temp\ipykernel_6848\130418045.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)
